In [ ]:
import numpy as np
import h5py
from scipy import signal
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import glob
import os
import mne
root_path = "//172.16.88.200/private/2221012/MIOnly_FTP_EEG Dataset and OpenBMI Toolbox for Three BCI Paradigms"
file_paths = glob.glob(root_path + "/pres/*.set")
fs = 500
ch = 14 #C4

In [ ]:
finfo_list = []
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    fns = file_name.split("_")
    session = int(fns[0][4:])
    subject = int(fns[1][4:])
    finfo_list.append((file_path,subject,session))

'Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'FC3', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CPz', 'CP4', 'P1', 'P2', 'POz', 'FT9', 'FTT9h', 'TTP7h', 'TP7', 'TPP9h', 'FT10', 'FTT10h', 'TPP8h', 'TP8', 'TPP10h', 'F9', 'F10', 'AF7', 'AF3', 'AF4', 'AF8', 'PO3', 'PO4'

## Preview

In [ ]:
preview_path,_,_ = finfo_list[0]
x = mne.io.read_epochs_eeglab(preview_path).get_data(item=["left","right"])
x.shape

### 標準化

In [ ]:
dx = x[0,:,:]
dx = StandardScaler().fit_transform(dx.T).T
#test: np.array([[1,2,3],[10,20,30]]),StandardScaler().fit_transform(np.array([[1,2,3],[10,20,30]]).T).T

### パワースペクトルに変換

In [ ]:
# パワースペクトルへの変換
freq, px = signal.periodogram(dx, fs=fs)
print(len(freq))
plt.plot(freq,np.mean(px,axis=0))
plt.xlim((2,30))
plt.ylim((0,0.050))

## まとめて処理(trainのみ)

In [ ]:
from eeghdf import EEGHDFUpdater
updater = EEGHDFUpdater(hdf_path=root_path+"/ex.h5",
                        fs=250,
                        lables=["left","right"])
for path,_,_ in finfo_list:
    ### 前処理 一旦保留
    #x = StandardScaler().fit_transform(x.T).T #標準化
    # #_, x = signal.periodogram(x, fs=fs/down_rate) #パワースペクトル
    
    ###
    updater.add_eeglab(path)